# **Wikipedia spanish language model**

In [ ]:
!pip install fastai2

In [ ]:
!pip show fastai2

In [ ]:
#!pip install torch===1.6.0 torchvision===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
from google.colab import drive, files
import os
import re
from pathlib import Path
from fastai2.text.all import *
import dill as pickle

In [ ]:
drive.mount('/content/drive/')

### Data preparation (Files with results in drive)

Connecting drive, train and test set organization, tokenization, numericalization and creation of LMDataLoader

In [ ]:
#to not show cell output-- too long
%%capture
!unzip -q "/content/drive/My Drive/Colab Notebooks/TLAC/wiki_art.zip"

In [ ]:
def set_organization(dset):
  
  path="/content/wiki_art/{}".format(dset)

  for filename in os.listdir(path):
    os.rename(os.path.join(path, filename), os.path.join(path, re.sub('[^A-Za-z0-9]+(?![^.]*$)', '',filename)))

  n_files = len(list(os.listdir(path)))
  !ls $path
  print("elements_{}_set={} ".format(dset,n_files))


In [ ]:
%%capture
set_organization("train")

In [ ]:
%%capture
set_organization("test")

In [ ]:
path = Path('/content/wiki_art')
path.ls()

In [ ]:
texts = get_files(path, extensions=['.txt'], folders=['train', 'test'])

In [ ]:
def read_file(f): return L(f.read().split(' '))

In [ ]:
splits = RandomSplitter(valid_pct=0.1)(texts)
tfms = [Tokenizer.from_folder(path), Numericalize()]
dsets = Datasets(texts, [tfms], splits=splits, dl_type=LMDataLoader)

In [ ]:
bs,sl=32,80
dbunch_lm = dsets.dataloaders(bs=bs, seq_len=sl, val_bs=bs)

In [ ]:
dbunch_lm.show_batch()

In [ ]:
pickle.dump(dsets, open(f'/content/dsets.pkl','wb'))

In [ ]:
!zip -r "/content/wiki_art_tok.zip" "/content/wiki_art_tok/"

In [ ]:
!mv "/content/dsets.pkl" "/content/drive/My Drive/Colab Notebooks/TLAC"

In [ ]:
!mv "/content/wiki_art_tok.zip" "/content/drive/My Drive/Colab Notebooks/TLAC"

##Model



Importing folder with tokens from google drive, creation of dataloader and the model architecture

In [ ]:
!unzip -q "/content/drive/My Drive/Colab Notebooks/TLAC/wiki_art_tok.zip"

In [ ]:
%%capture
!unzip /content/wiki_art_tok.zip

In [ ]:
!mv "/content/content/wiki_art_tok" "/content/"

In [ ]:
dsets = pickle.load(open(f'/content/drive/My Drive/Colab Notebooks/TLAC/dsets.pkl','rb'))

In [ ]:
bs,sl=32,80
dbunch_lm = dsets.dataloaders(bs=bs, seq_len=sl, val_bs=bs)

In [ ]:
opt_func = partial(Adam, wd=0.1)
learn = language_model_learner(dbunch_lm, AWD_LSTM, opt_func=opt_func, metrics=accuracy)
learn = learn.to_fp16(clip=0.1)

###Training (Files with results in drive)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(1, 7e-2, moms=[0.8,0.7,0.8])

In [ ]:
learn.save('wm1')

In [ ]:
!zip -r /content/models.zip /content/models

In [ ]:
files.download("/content/models.zip")

In [ ]:
%%capture
!unzip -q "/content/drive/My Drive/Colab Notebooks/TLAC/models.zip"

In [ ]:
!mv "/content/content/models" "/content/"

In [ ]:
learn.load('wm1')

In [ ]:
learn.validate() #Return the calculated loss and the metrics of the model